## Natual Language Processing

- Import csv from scraping notebook
- Binarize category column
- combine title & body
- stem and clean columns into engineered features

In [73]:
# laod libraries
import pandas as pd
import numpy as np
import regex as re
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
# load in data from previous notebook
df = pd.read_csv('./data/raw_dict.csv', index_col =0)

In [5]:
# making my category column binary
df['cat_nums'] = df['category'].map({'conspiracy': 0,
                                     'history': 1})

In [45]:
# make new column with body and text
df['pre_combined'] = df['text'].apply(lambda x: x if type(x)== str else '')
df['combined'] =  df['title'] + df['pre_combined']
# make new column for body or not
df['binary_text'] = df['text'].apply(lambda x: 1 if type(x)== str else 0)

In [46]:
# drop unecessary columns
df.drop(columns = ['pre_combined', 'category', 'text'], inplace=True) 

In [57]:
df.head()

,title,cat_nums,combined,binary_text
0,George Popadopoulos Judiciary Committee Transc...,0,George Popadopoulos Judiciary Committee Transc...,0
1,Scientists Will Spray Particles Into the Sky t...,0,Scientists Will Spray Particles Into the Sky t...,0
2,We Are Change confronted Joe Biden in 2007 abo...,0,We Are Change confronted Joe Biden in 2007 abo...,0
3,Jeff Sessions swats creepy Uncle Joe's hands away,0,Jeff Sessions swats creepy Uncle Joe's hands away,0
4,NXIVM cultist (🍕 gate) admits to enslaving wom...,0,NXIVM cultist (🍕 gate) admits to enslaving wom...,0


In [59]:
tokenizer = RegexpTokenizer(r'\w+')
p_stemmer = PorterStemmer()

In [60]:
# code from class NLP I
def review_to_words(raw_review):
    # 1. Remove non-letters.
    letters_only = re.sub("[^a-zA-Z]", " ", raw_review)
    
    #3. tokenize the script
    tokens = tokenizer.tokenize(letters_only)
    
    #4. stem the words
    clean_stems = [p_stemmer.stem(w) for w in tokens]
    
    # 5. Join the words back into one string separated by space 
    return(" ".join(clean_stems))

In [67]:
# create new column with transformed title
stem_title = [review_to_words(text) for text in df['title']]
df['stemmed_title']= stem_title

In [64]:
# create new column with transformed combined data
stem_combined = [review_to_words(text) for text in df['combined']]
df['stemmed_combined']= stem_combined

In [95]:
# EDA on dt classifier to know what to gridsearch for modeling
cvec = CountVectorizer(stop_words = 'english')
dt = DecisionTreeClassifier()
X = df['stemmed_combined']
y = df['cat_nums']
X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                   random_state=25,
                                                   stratify=y)
X_tr_cv = cvec.fit_transform(X_train)
dt.fit(X_tr_cv, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [100]:
# max depth
print(dt.tree_.max_depth)
print(dt.tree_.n_features)

48
9957


In [83]:
# gathered data from running the above code
# will use to hone features on gridsearch
# title             n_features: 4939,  depth: 206
# stemmed_title     n_features: 3757,  depth: 154
# combined          n_features: 14874, depth: 63
# stemmed_combined  n_features: 9957,  depth: 48

559

In [98]:
#load out file ready to model
df.to_csv('./data/final_df.csv')